In [ ]:
# interact with LLM via API authn

import os
from openai import OpenAI
from dotenv import load_dotenv

load_dotenv()
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

try:
    # This will tell you if your API key is valid and show account info
    models = client.models.list()
    print(f"Available models: {len(models.data)}")

    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": "You are a helpful DevOps assistant."},
            {"role": "user", "content": "Explain how CI/CD works in Azure DevOps."}
        ]
    )
    print(response.choices[0].message.content)
except Exception as e:
    print(e)

In [ ]:
# plot sigmoid function

import numpy as np
import matplotlib.pyplot as plt

# Define the function
x = np.linspace(-10, 10, 400)
y = 1 / (1 + np.exp(-x))

# Plot
plt.figure(figsize=(8, 5))
plt.plot(x, y, linewidth=2)
plt.title(r"Plot of $f(x) = \frac{1}{1 + e^{-x}}$ (Sigmoid Function)")
plt.xlabel("x")
plt.ylabel("f(x)")
plt.grid(True)
plt.show()

In [ ]:
# Fine-tuning

import torch
import evaluate
import hf_xet
import numpy as np
from datasets import load_dataset
import pandas as pd
from transformers import GPT2Tokenizer, GPT2ForSequenceClassification, TrainingArguments, Trainer

try:
    dataset = load_dataset('mteb/tweet_sentiment_extraction')
    df_train = pd.DataFrame(dataset['train'])
    df_test = pd.DataFrame(dataset['test'])


    tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
    tokenizer.pad_token = tokenizer.eos_token
    def tokenize(examples):
        return tokenizer(examples['text'], padding='max_length', truncation=True)
    tokenized_datasets = dataset.map(tokenize, batched=True)
    small_train_dataset = tokenized_datasets['train'].shuffle(seed=42).select(range(1000))
    small_eval_dataset = tokenized_datasets['test'].shuffle(seed=42).select(range(1000))

    model = GPT2ForSequenceClassification.from_pretrained('gpt2', num_labels=3)

    metric = evaluate.load('accuracy')
    def compute_metrics(eval_pred):
        logits, labels = eval_pred
        predictions = np.argmax(logits, axis=-1)
        return metric.compute(predictions=predictions, references=labels)

    training_args = TrainingArguments(
        output_dir="test_trainer",
        #evaluation_strategy="epoch",
        per_device_train_batch_size=1,  # Reduce batch size here
        per_device_eval_batch_size=1,    # Optionally, reduce for evaluation as well
        gradient_accumulation_steps=4
    )
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=small_train_dataset,
        eval_dataset=small_eval_dataset,
        compute_metrics=compute_metrics,
    )
    trainer.train()
    trainer.evaluate()
except Exception as e:
    print(e)